In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Diversification of the portfolio

## Tickers

In [5]:
with open('tickers.txt', 'r') as file:
    lines = file.readlines()
    
portfolio = {}

for line in lines:
    key, value = line.strip().split('=', 1)
    list_name = key.strip()
    items = value.strip().strip('[]').replace("'", "").split(', ')
    portfolio[list_name] = items

portfolio_columns = "stocks efts coins crypto futures".split(' ')

### Historical data

In [6]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

`yfinance.download` has the column `Adj Close`, which is not present in any `Ticker` method, therefore it is extracted to begin with.

In [7]:
now = datetime.now()
start = now - timedelta(days=59)

stock = 'BYDDY'

df = yf.download(stock, end=now.strftime('%Y-%m-%d'), start=start.strftime('%Y-%m-%d'), interval='5m')['Adj Close']
display(df)

[*********************100%%**********************]  1 of 1 completed


Datetime
2024-01-08 09:30:00-05:00    52.756001
2024-01-08 09:35:00-05:00    52.661999
2024-01-08 09:40:00-05:00    52.750000
2024-01-08 09:45:00-05:00    52.590000
2024-01-08 09:50:00-05:00    52.598000
                               ...    
2024-03-04 15:35:00-05:00    48.250000
2024-03-04 15:40:00-05:00    48.299999
2024-03-04 15:45:00-05:00    48.189999
2024-03-04 15:50:00-05:00    48.249500
2024-03-04 15:55:00-05:00    48.240002
Name: Adj Close, Length: 3021, dtype: float64

`yfinance.Tickers.history` provides the rest of the market historical data. It is then innerly joined with the previous DataFrame on `Datetime` 

In [8]:
stock_ticker = yf.Ticker(stock)
stock_hist = stock_ticker.history(end=now.strftime('%Y-%m-%d'), start=start.strftime('%Y-%m-%d'), interval='5m')
stock_hist.reset_index(inplace=True)
stock_hist.set_index('Datetime', inplace=True)
stock_hist = pd.concat([stock_hist, df], axis=1, join='inner')
stock_hist.reset_index(inplace=True)
stock_hist.drop('Datetime', axis=1, inplace=True)
stock_hist.sample(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Adj Close
175,52.525002,52.538502,52.500000,52.538502,6595,0.0,0.0,52.538502
2570,48.990002,48.990002,48.880001,48.910000,2981,0.0,0.0,48.910000
731,49.570000,49.599998,49.570000,49.599998,4830,0.0,0.0,49.599998


In [11]:
import numpy as np 

def historical(ticker):
    now = datetime.now()
    start = now - timedelta(days=58)

    stock = ticker

    df = yf.download(stock, end=now.strftime('%Y-%m-%d'), start=start.strftime('%Y-%m-%d'), interval='5m', progress=False)['Adj Close']
    stock_ticker = yf.Ticker(stock)
    stock_hist = stock_ticker.history(end=now.strftime('%Y-%m-%d'), start=start.strftime('%Y-%m-%d'), interval='5m');
    stock_hist.reset_index(inplace=True)
    stock_hist.set_index('Datetime', inplace=True)
    stock_hist = pd.concat([stock_hist, df], axis=1)
    stock_hist['returns'] = stock_hist['Adj Close'].pct_change()
    stock_hist['s'] = stock_hist.index.map(pd.Timestamp.timestamp)
    stock_hist['day sin'] = np.sin(stock_hist['s']*(2*np.pi / 60/60/24))
    stock_hist.reset_index(inplace=True)
    stock_hist.drop(['s', 'Datetime'], axis=1, inplace=True)
    stock_hist.ffill(inplace=True)
    stock_hist.dropna(inplace=True)
    
    return stock_hist.to_dict()

Apply it to all the stocks

In [12]:
from tqdm import tqdm

portfolio_hist = {}
pbar = tqdm(total=sum([len(x) for x in portfolio.values()]))
columns = iter(portfolio.keys())
for eq in portfolio.values():
    aux = pd.DataFrame()
    for stck in eq:
        df = historical(stck)
        aux[stck] = df
        pbar.update(1)
    multi_indexed_df = pd.concat({(i, j): pd.Series(v) for i, d in aux.to_dict().items() for j, v in d.items()}, axis=0)
    hist = pd.DataFrame(multi_indexed_df.unstack().transpose())
    #hist = hist.swaplevel(0, 1, axis=1)
    hist = hist.sort_index(axis=1)
    portfolio_hist[next(columns)] = hist.dropna() 
pbar.close()  

100%|██████████| 31/31 [00:13<00:00,  2.24it/s]


In [13]:
data = portfolio_hist['stocks']['AMD']['day sin']

In [17]:
portfolio.keys()

dict_keys(['stocks', 'efts', 'coins', 'crypto', 'futures'])